In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
def get_window_name(row):
    from audio_stack.parameters import WINDOW_TYPES
    try:
        window_number = int(row.appendix.replace("_window", ""))
    except:
        window_number = 3
    return WINDOW_TYPES[window_number]

In [ ]:
#exp_name = '2021_02_19_windows';
exp_name = '2021_06_09_stepper';
fname = f"../experiments/{exp_name}/all_data.pkl"

try:
    df_all = pd.read_pickle(fname)
    print("read", fname)
except Exception as e:
    print(e)
    print("Error: run wall_analysis.py to parse experiments.")
df_all.iloc[:, :8]

# plot_spectrograms(df_freq)

In [ ]:
from plotting_tools import plot_raw_signals, save_fig, add_colorbar
from dataset_parameters import kwargs_datasets
from pandas_utils import filter_by_dict

from data_collector import DataCollector
from frequency_analysis import get_index_matrix

mag_thresh = 1.0 # tested for 25_* and 23_* dataset
ymin = 1e-4
ymax = 3e2
degree = 0
verbose = False

distance = df_all.iloc[0].distance
print('using unique distance', distance)

chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    "mic_type": "audio_deck",
    #"appendix": "_window2"
}
df_filtered = filter_by_dict(df_all, chosen_dict)
print("plotting:\n", df_filtered.iloc[:, :8])

for mic_type, df_mic_types in df_filtered.groupby("mic_type"):
    if mic_type == "measurement":
        mic = 0
    else:
        mic = 1
        
    for j, (i_row, row) in enumerate(df_mic_types.iterrows()):
        if (mic_type == "measurement") and (row.snr == 1):  # this is no "real snr"
            continue
            
        window = get_window_name(row)

        label = f"motors:{row.motors}, window:{window:s}"

        data_collector = DataCollector.init_from_row(exp_name, row)
        spec_all = np.abs(data_collector.current_spectrogram)
        freqs = data_collector.current_freqs
        
        # plotting
        fig_raw, ax = plot_raw_signals(spec_all, freqs, mic_idx=mic)
        fig_raw.set_size_inches(5, 5)
        ax.set_ylim(ymin, ymax)
        ax.set_xlim(2000, 3500)
        ax.set_title(label)
        save_fig(
            fig_raw, f"plots/{exp_name}_{row.motors}{row.appendix}_raw.png", extension="png"
        )
        break

## interpolation study

current status: interpolation is not helping much because we need to consider the wider shape of the peak (due to windowing) in the pipeline somewhere. The fit_peak function might be a better solution but it is also not quite working yet. 

In [ ]:
import time
from frequency_analysis import interpolate_peak, fit_peak

mic_idx = 0
chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    "mic_type": "audio_deck",
    "appendix":"_window2",
    "motors":"all45000"
}
df_filtered = filter_by_dict(df_all, chosen_dict)
assert len(df_filtered) == 1
row = df_filtered.iloc[0]
data_collector = DataCollector.init_from_row(exp_name, row)

spec = data_collector.current_spectrogram
freqs = data_collector.current_freqs

n_extra = 12
time_range = list(range(282, 291))
fig, ax = plt.subplots()
plot_range = list(
    range(time_range[0]-n_extra, time_range[0]) 
) + time_range + list(
    range(time_range[-1]+1, time_range[-1]+n_extra+1) 
)
spec_plot =  np.abs(spec[:, mic_idx, plot_range])
freqs_valid = np.any(spec_plot, axis=1)
spec_plot = spec_plot[freqs_valid]
im = ax.pcolorfast(plot_range, freqs[freqs_valid], spec_plot[:-1, :-1])
add_colorbar(fig, ax , im)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)
fig.set_size_inches(10, 5)

for t_idx in time_range:
    spec_slice = spec[:, mic_idx, t_idx]

    max_without = np.max(np.abs(spec_slice))
    print(f'magniutde estimate without fitting: {max_without:.2f}')

    # use full slice
    max_val, max_freq = interpolate_peak(spec_slice, freqs)#, ax=ax)
    
    #print(f'magniutde estimate with interpolation with all points: {max_val:.2f}')
    phase = np.angle(spec_slice)
    axs[0].plot(freqs, np.abs(spec_slice), label=t_idx)
    axs[1].plot(freqs, phase, label=t_idx)
    #ax.axvline(max_freq, color='C1')
    axs[0].set_xlim(max_freq-500, max_freq+500)
    axs[0].set_xlim(5400, 5700)

    # use slice around max
    bin_max = np.argmax(np.abs(spec_slice))
    n_points = 2
    range_max = range(bin_max-n_points, bin_max+n_points)
    spec_smaller = spec_slice[range_max]
    freq_smaller = freqs[range_max]

    max_val_smaller, max_freq_smaller = interpolate_peak(spec_smaller, freq_smaller, pad_factor=2)
    #print(f'magniutde estimate with interpolation with {n_points} points: {max_val_smaller:.2f}')
    #ax.plot(freqs, np.abs(spec_slice))
    #ax.axvline(max_freq_smaller, color='C2')
    #ax.set_xlim(max_freq_smaller-500, max_freq_smaller+500)

    max_peak, p = fit_peak(np.abs(spec_slice))
    print(f'magniutde estimate with peak fitting: {max_peak:.2f}')
axs[0].legend()

In [ ]:
chosen_dict = {
    "degree": degree,
    "distance": distance,
    "mic_type": "audio_deck",
}
df_filtered = filter_by_dict(df_all, chosen_dict)
df_filtered = df_filtered[df_filtered.appendix.isin([f"_window{i}" for i in range(3)])]

if mic_type == "measurement":
    mic = 0
else:
    mic = 1

n_rows = len(df_filtered.appendix.unique())
fig_slice, ax_slices = plt.subplots(2, n_rows, sharex=True, sharey=True, squeeze=False)
fig_slice.set_size_inches(5 * n_rows, 10)

for j, (appendix, df) in enumerate(df_filtered.groupby("appendix", sort=True)):
    print('appendix', appendix)
    for i, (motors, df_row) in enumerate(df.groupby("motors", sort=True)):
        print('motors', motors)
        
        assert len(df_row) == 1, df_row.iloc[:, :10]
        row = df_row.iloc[0]

        t0 = time.time()
        
        data_collector = DataCollector.init_from_row(exp_name, row, interpolation='')
        t1 = time.time()
        data_collector_lagrange = DataCollector.init_from_row(exp_name, row, interpolation='lagrange')
        t2 = time.time()
        data_collector_quadratic = DataCollector.init_from_row(exp_name, row, interpolation='quadratic')
        t3 = time.time()
        print(f'times with interpolation:\n none:{t1-t0:.2e}s \n zero-pad:{t2-t1:.2e}s \n quadratic:{t3-t2:.2e}')

        for idx, dc in enumerate([data_collector, data_collector_lagrange, data_collector_quadratic]):
            df_mic = dc.df[dc.df.mic == mic]
            
            label = f"interpolation {dc.interpolation}"
            stds = (
                df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanstd).values
            )
            medians = (
                df_mic.groupby("frequency", sort=True)
                .magnitude.apply(np.nanmedian)
                .values
            )
            xs = np.sort(df_mic.frequency.unique())
            ax_slices[i, j].plot(xs, medians, label=label)
            #ax_slices[i, j].plot(xs, stds, label=label)
            
            #stds = np.r_[[np.zeros(len(medians))], [10 * stds]]
            #dx_mean = np.mean(xs[:-1]-xs[1:])
            #ax_slices[i, j].errorbar(
            #    x=xs+idx*dx_mean/5,
            #    y=medians,
            #    yerr=stds,
            #    label=label,
            #    marker="o",
            #    markersize=4.0,
            #    lw=2.0,
            #    alpha=1-idx*0.3
            #)
        window = get_window_name(row)
        title = f"motors:{row.motors}, window:{window:s}"
        ax_slices[i, j].grid(which="both")
        ax_slices[i, j].set_yscale('log')
        ax_slices[i, j].set_ylim(1.0, 30)
        ax_slices[i, j].set_title(title)
        ax_slices[i, j].legend(loc='lower right')
    title = f"window:{window}"
    #ax_slices[i, j].set_ylabel(f"motors {motors}")
[ax_slices[i, 0].set_ylabel(f"amplitude") for i in range(ax_slices.shape[0])]
[ax_slices[-1, i].set_xlabel("frequency [Hz]") for i in range(ax_slices.shape[1])]
save_fig(fig_slice, f"plots/{exp_name}_interpolations.png", extension="png")

In [ ]:
df = data_collector.df[data_collector.df.mic==0]
df

In [ ]:
plt.scatter(df.frequency, df.magnitude)